### 1. Data Schema(label & annotation schema)

#### 1.1 Label & annotation schema 분석

In [1]:
# label & annotation schema
import os, json
from collections import defaultdict, Counter

def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def summarize_schema(label_dir, person_id="0001", angles=("F","L","R"), preview_n=3):
    """
    1) angle별 idx(00~08) 파일들을 읽어서
       - idx마다 equipment/annotations 키 개수
       - idx마다 어떤 키가 들어있는지(미리보기)
    2) field가 어느 idx에 등장하는지(field->idx map) 생성
    """
    field2idx = defaultdict(lambda: defaultdict(Counter))  # field2idx[source][field][idx]+=1
    idx_profile = defaultdict(lambda: defaultdict(dict))   # idx_profile[angle][idx] = {eq_cnt, an_cnt, preview}

    for angle in angles:
        print(f"\n=== [{person_id} | angle={angle}] idx별 요약 ===")
        print("idx | file | #EQ | #AN | preview(keys)")
        print("-"*90)

        for idx in range(9):
            fn = f"{person_id}_03_{angle}_{idx:02d}.json"
            path = os.path.join(label_dir, fn)
            if not os.path.exists(path):
                print(f"{idx:>3} | {fn} | MISSING")
                continue

            data = load_json(path)
            eq = data.get("equipment", {}) or {}
            an = data.get("annotations", {}) or {}

            eq_keys = [k for k,v in eq.items() if v is not None]
            an_keys = [k for k,v in an.items() if v is not None]

            # field -> idx 기록
            for k in eq_keys:
                field2idx["equipment"][k][idx] += 1
            for k in an_keys:
                field2idx["annotations"][k][idx] += 1

            preview = (["EQ:"+k for k in eq_keys[:preview_n]] + ["AN:"+k for k in an_keys[:preview_n]])

            idx_profile[angle][idx] = {
                "eq_cnt": len(eq_keys),
                "an_cnt": len(an_keys),
                "preview": preview
            }

            print(f"{idx:>3} | {fn} | {len(eq_keys):>3} | {len(an_keys):>3} | {preview}")

    # -----------------------------
    # field -> idx 요약 출력
    # -----------------------------
    print("\n\n=== [Field -> idx mapping summary] ===")

    def show_field_map(source, topk=30):
        print(f"\n[{source}] 상위 field {topk}개 / 주로 등장하는 idx")
        # 전체 등장 빈도(모든 idx 합)
        total = []
        for field, idx_counts in field2idx[source].items():
            total_count = sum(idx_counts.values())
            total.append((field, total_count, idx_counts))
        total.sort(key=lambda x: x[1], reverse=True)

        for field, cnt, idx_counts in total[:topk]:
            best = idx_counts.most_common(3)  # 가장 많은 idx 3개
            print(f"- {field}: total={cnt} | idx_hits={best}")

    show_field_map("equipment", topk=20)
    show_field_map("annotations", topk=20)

    return idx_profile, field2idx

# 사용 예시
LABEL_DIR_0001 = r"D:\PyProject\datasets\skinAI\korea_skin_data\model_and_data\docker_image\skin_dataset\data_and_model\dataset\label\03\0001"
idx_profile, field2idx = summarize_schema(LABEL_DIR_0001, person_id="0001")


=== [0001 | angle=F] idx별 요약 ===
idx | file | #EQ | #AN | preview(keys)
------------------------------------------------------------------------------------------
  0 | 0001_03_F_00.json |   1 |   0 | ['EQ:pigmentation_count']
  1 | 0001_03_F_01.json |  15 |   2 | ['EQ:forehead_moisture', 'EQ:forehead_elasticity_R0', 'EQ:forehead_elasticity_R1', 'AN:forehead_pigmentation', 'AN:forehead_wrinkle']
  2 | 0001_03_F_02.json |   0 |   1 | ['AN:glabellus_wrinkle']
  3 | 0001_03_F_03.json |   8 |   1 | ['EQ:l_perocular_wrinkle_Ra', 'EQ:l_perocular_wrinkle_Rmax', 'EQ:l_perocular_wrinkle_Rt', 'AN:l_perocular_wrinkle']
  4 | 0001_03_F_04.json |   8 |   1 | ['EQ:r_perocular_wrinkle_Ra', 'EQ:r_perocular_wrinkle_Rmax', 'EQ:r_perocular_wrinkle_Rt', 'AN:r_perocular_wrinkle']
  5 | 0001_03_F_05.json |  16 |   2 | ['EQ:l_cheek_moisture', 'EQ:l_cheek_elasticity_R0', 'EQ:l_cheek_elasticity_R1', 'AN:l_cheek_pore', 'AN:l_cheek_pigmentation']
  6 | 0001_03_F_06.json |  16 |   2 | ['EQ:r_cheek_moisture', 'EQ

#### 1.2 Dataset 구성

In [4]:
import os
from tqdm import tqdm
from collections import defaultdict

# -------------------------
# 경로 설정
# -------------------------
BASE_PATH = r"D:\PyProject\datasets\skinAI\korea_skin_data\model_and_data\docker_image\skin_dataset\data_and_model\dataset"
IMG_ROOT = os.path.join(BASE_PATH, "img", "03")
LABEL_ROOT = os.path.join(BASE_PATH, "label", "03")

# -------------------------
# 1) 이미지 ↔ JSON 매핑 생성
# -------------------------
def build_image_label_mapping(label_root):
    mapping = defaultdict(list)

    for root, _, files in os.walk(label_root):
        for file in files:
            if not file.endswith(".json"):
                continue

            parts = file.split("_")
            if len(parts) < 4:
                continue

            person_id = parts[0]
            device_id = parts[1]
            angle = parts[2]

            base_key = f"{person_id}_{device_id}_{angle}"
            mapping[base_key].append(os.path.join(root, file))

    print(f"\n발견된 고유 이미지 키: {len(mapping)}개")
    return mapping

# -------------------------
# 2) dataset 생성
# -------------------------
def create_dataset(mapping, img_root):
    dataset = []

    for base_key, json_files in tqdm(mapping.items(), desc="데이터셋 생성"):
        try:
            person_id, device_id, angle = base_key.split("_")

            img_path = os.path.join(img_root, person_id, f"{base_key}.jpg")

            if not os.path.exists(img_path):
                continue

            sorted_json_files = sorted(
                json_files,
                key=lambda x: int(os.path.basename(x).split("_")[-1].split(".")[0])
            )

            if len(sorted_json_files) != 9:
                continue

            dataset.append({
                "image": img_path,
                "labels": sorted_json_files,
                "person_id": person_id,
                "angle": angle,
                "base_key": base_key
            })

        except Exception as e:
            print("에러:", e)
            continue

    print(f"\n최종 dataset 크기: {len(dataset)}")
    return dataset


# -------------------------
# 실행
# -------------------------
mapping = build_image_label_mapping(LABEL_ROOT)
dataset = create_dataset(mapping, IMG_ROOT)

# 샘플 확인
print("\n샘플 예시:")
sample = dataset[0]

print("이미지:", sample["image"])
print("JSON 개수:", len(sample["labels"]))
print("첫 JSON:", os.path.basename(sample["labels"][0]))
print("마지막 JSON:", os.path.basename(sample["labels"][-1]))


발견된 고유 이미지 키: 3216개


데이터셋 생성:  28%|█████████████████▏                                            | 894/3216 [00:00<00:00, 8939.94it/s]

에러: invalid literal for int() with base 10: '00-checkpoint'


데이터셋 생성: 100%|█████████████████████████████████████████████████████████████| 3216/3216 [00:00<00:00, 9084.76it/s]


최종 dataset 크기: 3215

샘플 예시:
이미지: D:\PyProject\datasets\skinAI\korea_skin_data\model_and_data\docker_image\skin_dataset\data_and_model\dataset\img\03\0001\0001_03_F.jpg
JSON 개수: 9
첫 JSON: 0001_03_F_00.json
마지막 JSON: 0001_03_F_08.json


#### 1.3 얼굴 위치(index 0 ~ 8)별 데이터 분석

In [7]:
import os, json
from collections import defaultdict, Counter

def load_json_safe(path):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except:
        return None

def global_idx_schema_stats(dataset, max_samples=None, preview_topk=15):
    """
    dataset(list): create_dataset 결과
    - idx(0~8)별로
      1) 파일 로드 성공률
      2) equipment/annotations에서 non-null 키 개수 평균
      3) 자주 등장하는 key TOPK
    """
    idx_read_ok = Counter()
    idx_read_fail = Counter()

    idx_eq_nonnull_cnt_sum = Counter()
    idx_an_nonnull_cnt_sum = Counter()
    idx_sample_cnt = Counter()  # idx별로 몇 번 관측했는지

    idx_eq_key_counter = defaultdict(Counter)  # idx -> Counter(field)
    idx_an_key_counter = defaultdict(Counter)  # idx -> Counter(field)

    checked = 0
    for s in dataset:
        for idx, jf in enumerate(s["labels"]):
            data = load_json_safe(jf)
            if data is None:
                idx_read_fail[idx] += 1
                continue

            idx_read_ok[idx] += 1
            idx_sample_cnt[idx] += 1

            eq = data.get("equipment", {}) or {}
            an = data.get("annotations", {}) or {}

            eq_keys = [k for k,v in eq.items() if v is not None]
            an_keys = [k for k,v in an.items() if v is not None]

            idx_eq_nonnull_cnt_sum[idx] += len(eq_keys)
            idx_an_nonnull_cnt_sum[idx] += len(an_keys)

            for k in eq_keys:
                idx_eq_key_counter[idx][k] += 1
            for k in an_keys:
                idx_an_key_counter[idx][k] += 1

        checked += 1
        if max_samples and checked >= max_samples:
            break

    print("\n=== [Global idx schema stats] ===")
    print(f"checked samples: {checked}\n")

    for idx in range(9):
        ok = idx_read_ok[idx]
        fail = idx_read_fail[idx]
        n = idx_sample_cnt[idx] if idx_sample_cnt[idx] > 0 else 1

        avg_eq = idx_eq_nonnull_cnt_sum[idx] / n
        avg_an = idx_an_nonnull_cnt_sum[idx] / n

        print(f"[idx={idx}] read_ok={ok} read_fail={fail} | avg_nonnull EQ={avg_eq:.2f} | avg_nonnull AN={avg_an:.2f}")

        top_eq = idx_eq_key_counter[idx].most_common(preview_topk)
        top_an = idx_an_key_counter[idx].most_common(preview_topk)

        if top_eq:
            print("  - equipment top keys:", [f"{k}({c})" for k,c in top_eq[:5]])
        else:
            print("  - equipment top keys: (none)")

        if top_an:
            print("  - annotations top keys:", [f"{k}({c})" for k,c in top_an[:5]])
        else:
            print("  - annotations top keys: (none)")

    return {
        "idx_read_ok": idx_read_ok,
        "idx_read_fail": idx_read_fail,
        "idx_eq_key_counter": idx_eq_key_counter,
        "idx_an_key_counter": idx_an_key_counter
    }

# 사용 예시:
stats = global_idx_schema_stats(dataset, max_samples=None, preview_topk=30) # preview_topk=10 출력 제한임 


=== [Global idx schema stats] ===
checked samples: 3215

[idx=0] read_ok=3215 read_fail=0 | avg_nonnull EQ=1.00 | avg_nonnull AN=0.00
  - equipment top keys: ['pigmentation_count(3215)']
  - annotations top keys: (none)
[idx=1] read_ok=3215 read_fail=0 | avg_nonnull EQ=15.00 | avg_nonnull AN=2.00
  - equipment top keys: ['forehead_moisture(3215)', 'forehead_elasticity_R0(3215)', 'forehead_elasticity_R1(3215)', 'forehead_elasticity_R2(3215)', 'forehead_elasticity_R3(3215)']
  - annotations top keys: ['forehead_pigmentation(3215)', 'forehead_wrinkle(3215)']
[idx=2] read_ok=3215 read_fail=0 | avg_nonnull EQ=0.00 | avg_nonnull AN=1.00
  - equipment top keys: (none)
  - annotations top keys: ['glabellus_wrinkle(3215)']
[idx=3] read_ok=3215 read_fail=0 | avg_nonnull EQ=8.00 | avg_nonnull AN=1.00
  - equipment top keys: ['l_perocular_wrinkle_Ra(3215)', 'l_perocular_wrinkle_Rmax(3215)', 'l_perocular_wrinkle_Rt(3215)', 'l_perocular_wrinkle_Rz=Rtm(3215)', 'l_perocular_wrinkle_Rp(3215)']
  - ann

In [8]:
# ============================================================
# 1. IDX → ROLE → 가능한 분석 항목 정의
# ============================================================

IDX_ANALYSIS_MAP = {

    0: {
        "role": "GLOBAL-얼굴 전체",
        "available_targets": {
            "pigmentation_count": {
                "task_type": ["regression", "classification"],
                "description": "얼굴 전체 색소 개수"
            }
        }
    },

    1: {
        "role": "FOREHEAD-이마",
        "available_targets": {
            "elasticity": {
                "task_type": ["regression"],
                "fields": "forehead_elasticity_R*, Q*"
            },
            "moisture": {
                "task_type": ["regression"],
                "fields": "forehead_moisture"
            },
            "pigmentation": {
                "task_type": ["classification"],
                "fields": "forehead_pigmentation"
            },
            "wrinkle": {
                "task_type": ["classification"],
                "fields": "forehead_wrinkle"
            }
        }
    },

    2: {
        "role": "GLABELLUS-미간",
        "available_targets": {
            "wrinkle": {
                "task_type": ["classification"],
                "fields": "glabellus_wrinkle"
            }
        }
    },

    3: {
        "role": "LEFT_PERIOCULAR-왼쪽 눈가 주름",
        "available_targets": {
            "wrinkle_score": {
                "task_type": ["regression"],
                "fields": "l_perocular_wrinkle_R*"
            },
            "wrinkle_grade": {
                "task_type": ["classification"],
                "fields": "l_perocular_wrinkle"
            }
        }
    },

    4: {
        "role": "RIGHT_PERIOCULAR- 오른쪽 눈가 주름",
        "available_targets": {
            "wrinkle_score": {
                "task_type": ["regression"],
                "fields": "r_perocular_wrinkle_R*"
            },
            "wrinkle_grade": {
                "task_type": ["classification"],
                "fields": "r_perocular_wrinkle"
            }
        }
    },

    5: {
        "role": "LEFT_CHEEK-왼쪽 볼",
        "available_targets": {
            "elasticity": {
                "task_type": ["regression"],
                "fields": "l_cheek_elasticity_*"
            },
            "moisture": {
                "task_type": ["regression"],
                "fields": "l_cheek_moisture"
            },
            "pore": {
                "task_type": ["classification"],
                "fields": "l_cheek_pore"
            },
            "pigmentation": {
                "task_type": ["classification"],
                "fields": "l_cheek_pigmentation"
            }
        }
    },

    6: {
        "role": "RIGHT_CHEEK-오른쪽 볼",
        "available_targets": {
            "elasticity": {
                "task_type": ["regression"],
                "fields": "r_cheek_elasticity_*"
            },
            "moisture": {
                "task_type": ["regression"],
                "fields": "r_cheek_moisture"
            },
            "pore": {
                "task_type": ["classification"],
                "fields": "r_cheek_pore"
            },
            "pigmentation": {
                "task_type": ["classification"],
                "fields": "r_cheek_pigmentation"
            }
        }
    },

    7: {
        "role": "LIP-입술",
        "available_targets": {
            "dryness": {
                "task_type": ["classification"],
                "fields": "lip_dryness"
            }
        }
    },

    8: {
        "role": "CHIN-턱",
        "available_targets": {
            "elasticity": {
                "task_type": ["regression"],
                "fields": "chin_elasticity_*"
            },
            "moisture": {
                "task_type": ["regression"],
                "fields": "chin_moisture"
            },
            "sagging": {
                "task_type": ["classification"],
                "fields": "chin_sagging"
            }
        }
    }
}
def print_idx_analysis_summary():
    print("\n===== 9개 부위별 분석 가능 항목 =====\n")

    for idx, info in IDX_ANALYSIS_MAP.items():
        print(f"[idx={idx}] ROLE = {info['role']}")
        for target, meta in info["available_targets"].items():
            print(f"  - {target}")
            print(f"      task_type : {meta['task_type']}")
            if "fields" in meta:
                print(f"      fields    : {meta['fields']}")
        print("-"*50)


print_idx_analysis_summary()


===== 9개 부위별 분석 가능 항목 =====

[idx=0] ROLE = GLOBAL-얼굴 전체
  - pigmentation_count
      task_type : ['regression', 'classification']
--------------------------------------------------
[idx=1] ROLE = FOREHEAD-이마
  - elasticity
      task_type : ['regression']
      fields    : forehead_elasticity_R*, Q*
  - moisture
      task_type : ['regression']
      fields    : forehead_moisture
  - pigmentation
      task_type : ['classification']
      fields    : forehead_pigmentation
  - wrinkle
      task_type : ['classification']
      fields    : forehead_wrinkle
--------------------------------------------------
[idx=2] ROLE = GLABELLUS-미간
  - wrinkle
      task_type : ['classification']
      fields    : glabellus_wrinkle
--------------------------------------------------
[idx=3] ROLE = LEFT_PERIOCULAR-왼쪽 눈가 주름
  - wrinkle_score
      task_type : ['regression']
      fields    : l_perocular_wrinkle_R*
  - wrinkle_grade
      task_type : ['classification']
      fields    : l_perocular_wrink

In [9]:
# ============================================
# Face-Level Multi-Head Output Definition
# (English + Korean Terminology)
# ============================================

FACE_MULTIHEAD_MAP = {

    "Elasticity(탄력성) Head": {
        "type": "Regression(회귀)",
        "outputs": {
            "forehead_elasticity(이마 탄력성)": "forehead_elasticity_R*, Q*",
            "l_cheek_elasticity(왼쪽 볼 탄력성)": "l_cheek_elasticity_*",
            "r_cheek_elasticity(오른쪽 볼 탄력성)": "r_cheek_elasticity_*",
            "chin_elasticity(턱 탄력성)": "chin_elasticity_*",
        }
    },

    "Pore(모공) Head": {
        "type": "Classification(분류)",
        "outputs": {
            "l_cheek_pore(왼쪽 볼 모공)": "l_cheek_pore",
            "r_cheek_pore(오른쪽 볼 모공)": "r_cheek_pore",
        }
    },

    "Pigmentation(색소침착) Head": {
        "type": "Mixed(혼합: Regression + Classification)",
        "outputs": {
            "global_count(얼굴 전체 색소 개수)": "pigmentation_count",
            "forehead_pigmentation(이마 색소침착)": "forehead_pigmentation",
            "l_cheek_pigmentation(왼쪽 볼 색소침착)": "l_cheek_pigmentation",
            "r_cheek_pigmentation(오른쪽 볼 색소침착)": "r_cheek_pigmentation",
        }
    },

    "Moisture(수분) Head": {
        "type": "Regression(회귀)",
        "outputs": {
            "forehead_moisture(이마 수분)": "forehead_moisture",
            "l_cheek_moisture(왼쪽 볼 수분)": "l_cheek_moisture",
            "r_cheek_moisture(오른쪽 볼 수분)": "r_cheek_moisture",
            "chin_moisture(턱 수분)": "chin_moisture",
        }
    },

    "Wrinkle(주름) Head": {
        "type": "Classification(분류)",
        "outputs": {
            "forehead_wrinkle(이마 주름)": "forehead_wrinkle",
            "glabellus_wrinkle(미간 주름)": "glabellus_wrinkle",
            "l_periocular_wrinkle(왼쪽 눈가 주름)": "l_perocular_wrinkle",
            "r_periocular_wrinkle(오른쪽 눈가 주름)": "r_perocular_wrinkle",
        }
    },

    "Lip Dryness(입술 건조도) Head": {
        "type": "Classification(분류)",
        "outputs": {
            "lip_dryness(입술 건조도)": "lip_dryness",
        }
    },

    "Chin Sagging(턱 처짐) Head": {
        "type": "Classification(분류)",
        "outputs": {
            "chin_sagging(턱 처짐)": "chin_sagging",
        }
    }
}


# ============================================
# 출력 함수
# ============================================

def print_face_multihead_summary():
    print("\n===== Face-Level Multi-Head Structure (얼굴 전역 멀티헤드 구조) =====\n")

    for head_name, head_info in FACE_MULTIHEAD_MAP.items():
        print(f"🟢 {head_name}")
        print(f"   task_type : {head_info['type']}")
        print("   outputs:")

        for output_name, field in head_info["outputs"].items():
            print(f"     - {output_name}")
            print(f"         measured_by : {field}")

        print("-" * 70)


# 실행
print_face_multihead_summary()


===== Face-Level Multi-Head Structure (얼굴 전역 멀티헤드 구조) =====

🟢 Elasticity(탄력성) Head
   task_type : Regression(회귀)
   outputs:
     - forehead_elasticity(이마 탄력성)
         measured_by : forehead_elasticity_R*, Q*
     - l_cheek_elasticity(왼쪽 볼 탄력성)
         measured_by : l_cheek_elasticity_*
     - r_cheek_elasticity(오른쪽 볼 탄력성)
         measured_by : r_cheek_elasticity_*
     - chin_elasticity(턱 탄력성)
         measured_by : chin_elasticity_*
----------------------------------------------------------------------
🟢 Pore(모공) Head
   task_type : Classification(분류)
   outputs:
     - l_cheek_pore(왼쪽 볼 모공)
         measured_by : l_cheek_pore
     - r_cheek_pore(오른쪽 볼 모공)
         measured_by : r_cheek_pore
----------------------------------------------------------------------
🟢 Pigmentation(색소침착) Head
   task_type : Mixed(혼합: Regression + Classification)
   outputs:
     - global_count(얼굴 전체 색소 개수)
         measured_by : pigmentation_count
     - forehead_pigmentation(이마 색소침착)
         measure